In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from utils.match import match

In [7]:
class MultiBoxLoss(nn.Module):
    def __init__(self, jaccard_thresh=0.5, neg_pos=3, device='cpu'):
        super(MutilBoxLoss, self).__init__()
        self.jaccard_thresh = jaccard_thresh
        self.negpos_ratio = neg_pos
        self.device = device
        
    def forward(self, predictions, targets):
        '''
        損失関数の計算
        
        Parameters
        ----------
        predictions: SSD net 訓練時の出力(tuple)
            (loc=torch.Size([num_batch, 8732, 4]),
             conf=torch.Size([nun_batch, 8732, 21]),
             dbox_list=torch.Size([8732, 4]))
             
        targets: [num_batch, num_objs, 5]
             5は正解アノテーション情報[xmin, ymin, xmax, ymax, label_ind]
             
        Returns
        -------
        loss_l: テンソル
            locの損失の値
        loss_c: テンソル
            confの損失の値
        '''
        
        loc_data, conf_data, dbox_list = predictions
        
        num_batch = loc_data.size(0)
        num_dbox = loc_data.size(1)
        num_classes = conf_data.size(2)
        
        # 損失の計算に使用するものを格納する変数を作成
        # conf_t_label: 各DBoxに一番近い正解のBBoxのラベルを格納させる
        # loc_t: 各DBoxに一番近い正解のBBoxの位置情報を格納させる
        conf_t_label = torch.LongTensor(num_batch, num_dbox).to(self.device)
        loc_t = torch.Tensor(num_batch, num_dbox, 4).to(self.device)
        
        # loc_tとconf_t_labelに
        # DBoxと正解アノテーションtargetsをmatchさせた結果を上書き
        for idx in range(num_batch):
            # 現在のミニバッチの正解アノテーションのBBoxとラベルを取得
            truths = targets[idx][:, :-1].to(self.device) # BBox
            labels = targets[idx][:, -1].to(self.device)
            
            dbox = dbox_list.to(self.device)
            
            # 関数matchを実行し、loc_tとconf_t_labelの内容を更新
            # loc_t: 各DBoxに一番近い正解のBBoxの位置情報が上書きされる
            # conf_t_label: 各DBoxに一番近いBBoxのラベルが上書きされる
            # ただし、一番近いBBoxとのjaccard_overlapが0.5より小さい場合、正解BBoxのラベルは背景クラス0とする
            variance = [0.1, 0.2] # DBoxからBBoxへの補正計算の係数
            match(self.jaccard_thresh, truths, dbox, variance, labels, loc_t, conf_t_label, idx)
            
        # 位置の損失: loss_lを計算
        # Smooth_L1関数で損失を計算する。ただし、物体を発見したDBoxのオフセットのみを計算する
        pos_mask = conf_t_label > 0 # torch.Size([num_batch, 8732])
        
        pos_idx = pos_mask.unsqueeze(pos_mask.dim()).expand_as(loc_data)
        
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        
        loss_l = F.smooth_l1_loss(loc_p, loc_t, reduction='sum')
            
        # classの損失
        batch_conf = conf_data.view(-1, num_classes)
        
        loss_c = F.cross_entropy(batch_conf, conf_t_label.view(-1), reduction='none')
        
        # Hard Negative Mining
        num_pos = pos_mask.long().sum(1, keepdim=True) # ミニバッチゴトノブッタイクラスヨソクノカズ
        
        loss_c = loss_c.view(num_batch, -1) # torch.Size([num_batch, 8732])
        loss_c[pos_mask] = 0 # 物体を発見したDBoxは損失を0にする
        
        # Hard Negative Minig実施
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        
        num_neg = torch.clamp(num_pos*self.negpos_ratio, max=num_dbox)
        
        neg_mask = idx_rank < (num_neg).expand_as(idx_rank)
        
        pos_idx_mask = pos_mask.unsqueeze(2).expand_as(conf_data)
        neg_idx_mask = neg_mask.unsqueeze(2).expand_as(conf_data)
        
        conf_hnm = conf_data[(pos_idx_mask+neg_idx_mask).gt(0)].view(-1, num_classes)
        
        conf_t_label_hnm = conf_t_label[(pos_mask+neg_mask).gt(0)]
        
        loss_c = F.cross_entropy(conf_hnm, conf_t_label_hnm, reduction='sum')
        
        N = num_pos.sum()
        loss_l /= N
        loss_c /= N
        
        return loss_l, loss_c